In [1]:
import subprocess
import numpy as np
from Tools.evalute_model import evaluate_metrics
import os

In [37]:
perl_file_path='utils/convert_format.pl'

original_train_src='Eurlex/eurlex_train.txt'
original_test_src='Eurlex/eurlex_test.txt'

train_x_file_path='Parabel/dataset/train_feat.txt'
train_y_file_path='Parabel/dataset/train_label.txt'

test_x_file_path='Parabel/dataset/test_feat.txt'
test_y_file_path='Parabel/dataset/test_label.txt'

model_dir='Parabel/output'
pred_y_file_path='Parabel/output/output_scores.txt'
pred_train_y_file_path='Parabel/output/train_output_scores.txt'


# Converting data into required format

In [19]:
directory = os.path.dirname(train_x_file_path)


if not os.path.exists(directory):
    os.makedirs(directory)

command=["perl",perl_file_path,original_train_src,train_x_file_path,train_y_file_path]
converstion_train_output = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
print("Converted train data in required format")


command=["perl",perl_file_path,original_test_src,test_x_file_path,test_y_file_path]
conversation_test_output = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
print("Converted train data in the required format")

Converted train data in required format
Converted train data in the required format


In [19]:
# training the model
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

command = [
    "./Parabel/binaries/Linux/parabel_train",
    train_x_file_path,
    train_y_file_path,
    model_dir,
    "-T", "4",
    "-s", "0",
    "-t", "3",
    "-b", "1.0",
    "-c", "1.0",
    "-m", "100",
    "-tcl", "0.1",
    "-tce", "0",
    "-e", "0.0001",
    "-n", "20",
    "-k", "0",
    "-q", "0"
]



train_result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
print(train_result.stdout)
print("Training complete")

tree 0 training startedtree 
1 training started
tree 0 training startedtree 
1 training started
tree 2 training started
node 0
node 0
node 0
node 0
node 100
node 100
node 100
tree 0 training completed
tree 2 training completed
tree 1 training completed
Training time: 0.0206362 hr

Training complete


In [23]:
# testing the model
command = [
    "./Parabel/binaries/Linux/parabel_predict",     # Ensure binary is compiled and accessible
    test_x_file_path,
    model_dir,
    pred_y_file_path,
    "-T", "4",               # Number of threads
    "-s", "0",               # Start tree
    "-t", "3",               # Number of trees
    "-B", "10",              # Beam width
    "-q", "0"                # Verbosity
]

test_result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
print(test_result.stdout)
print("Testing complete")

tree 0 predicting started
tree 1 predicting started
tree 2 predicting started
node 0
node 0
node 0
node 100
node 100
node 100
tree 0 predicting completed
tree 1 predicting completed
tree 2 predicting completed
prediction time: 1.25592 ms/point
model size: 0.0259221 GB

Testing complete


In [24]:
avg_precision, avg_ndcg = evaluate_metrics(test_y_file_path, pred_y_file_path)


print("Evaluation Metrics (k = 1 to 10):\n")
print(f"{'k':<4} {'Precision@k':<10} {'nDCG@k':<10}")
print("-" * 34)

for k in range(1, 11):
    print(f"{k:<4} {avg_precision[k]*100:<10.2f} {avg_ndcg[k]*100:<10.2f}")

Evaluation Metrics (k = 1 to 10):

k    Precision@k nDCG@k    
----------------------------------
1    82.17      82.17     
2    75.18      76.78     
3    68.81      72.28     
4    63.02      68.76     
5    57.58      66.60     
6    52.56      66.08     
7    47.88      67.59     
8    43.78      68.86     
9    40.27      69.93     
10   37.25      70.81     


In [27]:
# testing the model on train dataset to be used by NN to create surrogate model

command = [
    "./Parabel/binaries/Linux/parabel_predict",     # Ensure binary is compiled and accessible
    train_x_file_path,
    model_dir,
    pred_train_y_file_path,
    "-T", "4",               # Number of threads
    "-s", "0",               # Start tree
    "-t", "3",               # Number of trees
    "-B", "10",              # Beam width
    "-q", "0"                # Verbosity
]

test_result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
print(test_result.stdout)
print("Testing on training dataset complete")

tree tree 10 predicting started
tree tree 10 predicting started
tree tree 10 predicting started
tree 2 predicting started
node 0
node 0
node 0
node 100
node 100
node 100
tree 2 predicting completed
tree 1 predicting completed
tree 0 predicting completed
prediction time: 1.34458 ms/point
model size: 0.0259221 GB

Testing on training dataset complete
